In [52]:
import ast
import glob

In [55]:
# Read all *.txt files in the ./tweet_jsons/ directory and print every tweet out
for filepath in glob.iglob('./tweet_jsons/*.txt'):
    print(filepath)
    f = open(filepath, "r")
    Lines = f.readlines() # Read every file line by line, 1 line contains one tweet object, see the files for more info
    
    print("\n")
    
    for line in Lines:
        print(json.loads(line)["content"]) # Load each line into json and print out the "content"
        
    print("\n")

./tweet_jsons/6BillionPeople.txt


@ someone on Twitter and give them a complement or just say thank you. You never know what that can do for someone. It’s time to start giving people their flowers while they’re alive.
Men stop competing with your woman! It’s not a competition, everything is appreciated and celebrated. Everything is a bonus, don’t compare unless it’s a plan to build. You being stronger, richer monetarily, smarter doesn’t make you better. Value comes in many ways
Make sure you compliment your significant other. Men love compliments just as much as women and gentlemen make sure you’re complimenting your woman daily. Text are cool but short term, saying it with your voice over the phone is better and saying it in person is the ultimate
Boost her confidence in every way possible.
Find your soul, then your soulmate unless you find someone who can help you find your soul for you!
If you have a #Clubhouse account click the link attached to this tweet. Let’s network and connec

@MarkuQendrim There's nothing wrong with the devices. Getting a better cold storage device doesn't remove this attack vector (social engineering and extortion via leaked PII).
@Crypto_hussler Heeeey hey heeeeeeey
PS. They're using a Segwit address. I don't know if I should be impressed or disgusted. Mixed feelings.
The emails are only "customized" to add each victim's particulars, but otherwise they are identical, down to the BTC and ETH addresses. 

For more information on the #LedgerHack and some helpful advice, watch this video series:

https://t.co/U7ks9UruoZ
@Doodlxr Well crap
If I received an email like this, I would ignore it. As scary as it is, it is a bluff. Still, it's a despicable tactic and very scary to receive. 

You can report this to your local FBI office - it is cyber-extortion and interstate fraud (at least). 

3/
The identical email has been sent to several people, which indicates it is a mass-bluff, likely being sent to everyone on the list whose shipping address wa